In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from datetime import date
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MultiLabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_array
import re

In [2]:
df1 = pd.read_parquet("df_final_compile.parquet")
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_colwidth', None)
print(df1.shape)
df1.head(3)

(5771, 38)


,id_movie,backdrop_path,original_language,overview,popularity_movie,poster_path,release_date,title,vote_average,vote_count,genre,gender_director,name_director,popularity_director,birthday_director,deathday_director,place_of_birth_director,movie_id,actor_name_1,actor_name_2,actor_name_3,actor_name_4,actor_name_5,actor_gender_1,actor_gender_2,actor_gender_3,actor_gender_4,actor_gender_5,actor_popularity_1,actor_popularity_2,actor_popularity_3,actor_popularity_4,actor_popularity_5,actor_birthday_1,actor_birthday_2,actor_birthday_3,actor_birthday_4,actor_birthday_5
0,11224,/rH0DPF7pB35jxLxKb3JRUgCrrnp.jpg,en,"D’un coup de baguette magique, la fée Marraine...",79.005,/nqSJwBdvG89uHRpDDdaAy5YhogZ.jpg,1950-02-22,Cendrillon,7.041,6685,"[Family, Fantasy, Animation, Romance]",2,Hamilton Luske,2.479,1903-10-16,1968-02-19,"Chicago, Illinois, USA",11224,Ilene Woods,Eleanor Audley,Verna Felton,Claire Du Brey,Rhoda Williams,1.0,1.0,1.0,1.0,1.0,5.250,7.009,8.036,4.592,2.626,1929-05-05,1905-11-19,1890-07-20,1892-08-30,1930-07-03
1,389,/qqHQsStV6exghCM7zbObuYBiYxw.jpg,en,Un jeune homme d'origine modeste est accusé du...,58.940,/bPImGSvZtG2tvsJ9bVLrIECRDnB.jpg,1957-04-10,12 Hommes en colère,8.500,8655,[Drama],2,Sidney Lumet,4.824,1924-06-25,2011-04-09,"Philadelphia, Pennsylvania, USA",389,Martin Balsam,John Fiedler,Lee J. Cobb,E.G. Marshall,Jack Klugman,2.0,2.0,2.0,2.0,2.0,9.780,7.985,10.268,9.454,4.698,1919-11-04,1925-02-03,1911-12-08,1914-06-18,1922-04-27
2,6844,/rI1u4JgtnAZC99wXX4gwro4yaxe.jpg,en,Évocation de la vie de Moïse sauvé à sa naissa...,56.194,/q0KM14O75n0h4324npmThHi56FG.jpg,1956-10-05,Les Dix Commandements,7.756,1603,"[Drama, History]",2,Cecil B. DeMille,6.705,1881-08-12,1959-01-21,"Ashfield, Massachusetts, USA",6844,Charlton Heston,Yul Brynner,Anne Baxter,Edward G. Robinson,Yvonne De Carlo,2.0,2.0,1.0,2.0,1.0,22.823,8.561,9.878,9.671,15.539,1923-10-04,1920-07-11,1923-05-07,1893-12-11,1922-09-01


In [3]:
save1 = pd.read_parquet("save1.parquet")
save1

,id_movie,Publique cible,Epoque,Sujet principal,Type
0,11224,enfant,moyen-âge,amour,dessin animé
1,389,adulte,année 1950-1960,drame,film d'auteur
2,6844,adulte,antiquité,drame,blockbuster
3,665,adulte,antiquité,drame,blockbuster
4,10882,enfant,moyen-âge,amour,dessin animé
...,...,...,...,...,...
2882,138843,adulte,années 1970-1980,peur,blockbuster
2883,76341,adulte,futuriste,postapocalyptique,blockbuster
2884,337167,adulte,époque actuelle,amour,blockbuster
2885,273481,adulte,époque actuelle,suspense,blockbuster


In [4]:
save2 = pd.read_parquet("save2.parquet")
save2

,id_movie,Publique cible,Epoque,Sujet principal,Type
0,73723,enfant,futuriste,animaux,dessin animé
1,315635,adulte,époque actuelle,super-héros,blockbuster
2,210577,adulte,époque actuelle,suspense,blockbuster
3,273477,adulte,futuriste,postapocalyptique,blockbuster
4,10192,enfant,époque actuelle,humour,dessin animé
...,...,...,...,...,...
2879,694234,adulte,époque actuelle,amour,film d'auteur
2880,714888,adulte,années 1970-1980,biopic,film d'auteur
2881,606954,adulte,époque actuelle,drame,film d'auteur
2882,614292,adulte,époque actuelle,drame,film d'auteur


In [5]:
save = pd.concat([save1, save2])
print(save.shape)
save

(5771, 5)


,id_movie,Publique cible,Epoque,Sujet principal,Type
0,11224,enfant,moyen-âge,amour,dessin animé
1,389,adulte,année 1950-1960,drame,film d'auteur
2,6844,adulte,antiquité,drame,blockbuster
3,665,adulte,antiquité,drame,blockbuster
4,10882,enfant,moyen-âge,amour,dessin animé
...,...,...,...,...,...
2879,694234,adulte,époque actuelle,amour,film d'auteur
2880,714888,adulte,années 1970-1980,biopic,film d'auteur
2881,606954,adulte,époque actuelle,drame,film d'auteur
2882,614292,adulte,époque actuelle,drame,film d'auteur


In [6]:
df = pd.merge(df1, save, how='left', on = ["id_movie", "id_movie"])
df

,id_movie,backdrop_path,original_language,overview,popularity_movie,poster_path,release_date,title,vote_average,vote_count,genre,gender_director,name_director,popularity_director,birthday_director,deathday_director,place_of_birth_director,movie_id,actor_name_1,actor_name_2,actor_name_3,actor_name_4,actor_name_5,actor_gender_1,actor_gender_2,actor_gender_3,actor_gender_4,actor_gender_5,actor_popularity_1,actor_popularity_2,actor_popularity_3,actor_popularity_4,actor_popularity_5,actor_birthday_1,actor_birthday_2,actor_birthday_3,actor_birthday_4,actor_birthday_5,Publique cible,Epoque,Sujet principal,Type
0,11224,/rH0DPF7pB35jxLxKb3JRUgCrrnp.jpg,en,"D’un coup de baguette magique, la fée Marraine...",79.005,/nqSJwBdvG89uHRpDDdaAy5YhogZ.jpg,1950-02-22,Cendrillon,7.041,6685,"[Family, Fantasy, Animation, Romance]",2,Hamilton Luske,2.479,1903-10-16,1968-02-19,"Chicago, Illinois, USA",11224,Ilene Woods,Eleanor Audley,Verna Felton,Claire Du Brey,Rhoda Williams,1.0,1.0,1.0,1.0,1.0,5.250,7.009,8.036,4.592,2.626,1929-05-05,1905-11-19,1890-07-20,1892-08-30,1930-07-03,enfant,moyen-âge,amour,dessin animé
1,389,/qqHQsStV6exghCM7zbObuYBiYxw.jpg,en,Un jeune homme d'origine modeste est accusé du...,58.940,/bPImGSvZtG2tvsJ9bVLrIECRDnB.jpg,1957-04-10,12 Hommes en colère,8.500,8655,[Drama],2,Sidney Lumet,4.824,1924-06-25,2011-04-09,"Philadelphia, Pennsylvania, USA",389,Martin Balsam,John Fiedler,Lee J. Cobb,E.G. Marshall,Jack Klugman,2.0,2.0,2.0,2.0,2.0,9.780,7.985,10.268,9.454,4.698,1919-11-04,1925-02-03,1911-12-08,1914-06-18,1922-04-27,adulte,année 1950-1960,drame,film d'auteur
2,6844,/rI1u4JgtnAZC99wXX4gwro4yaxe.jpg,en,Évocation de la vie de Moïse sauvé à sa naissa...,56.194,/q0KM14O75n0h4324npmThHi56FG.jpg,1956-10-05,Les Dix Commandements,7.756,1603,"[Drama, History]",2,Cecil B. DeMille,6.705,1881-08-12,1959-01-21,"Ashfield, Massachusetts, USA",6844,Charlton Heston,Yul Brynner,Anne Baxter,Edward G. Robinson,Yvonne De Carlo,2.0,2.0,1.0,2.0,1.0,22.823,8.561,9.878,9.671,15.539,1923-10-04,1920-07-11,1923-05-07,1893-12-11,1922-09-01,adulte,antiquité,drame,blockbuster
3,665,/sU3gFo7Gn67zjtKXIvzJ4omcUbH.jpg,en,"Judas Ben‐Hur, prince de Judée, retrouve son a...",56.369,/4KH7VM5YyGwxe4kLSNxix2PNTn6.jpg,1959-11-18,Ben-Hur,7.881,2747,"[History, Drama, Adventure, Action]",2,William Wyler,4.382,1902-07-01,1981-07-27,"Mülhausen, Alsace-Lorraine, German Empire [now...",665,Charlton Heston,Stephen Boyd,Hugh Griffith,Jack Hawkins,Haya Harareet,2.0,2.0,2.0,2.0,1.0,22.823,8.798,4.333,9.360,2.899,1923-10-04,1931-07-04,1912-05-30,1910-09-14,1931-09-20,adulte,antiquité,drame,blockbuster
4,10882,/1HWNGfaYB4GPwMg8piXYdQHWNuF.jpg,en,La sorcière Maléfique jette un sort à la princ...,46.057,/5GaczvmuUCxhwPNx896lo2OCIFh.jpg,1959-02-17,La Belle au bois dormant,6.900,5123,"[Fantasy, Animation, Romance, Family]",2,Clyde Geronimi,3.546,1901-06-12,1989-04-24,"Chiavenna, Lombardy, Italy",10882,Mary Costa,Bill Shirley,Eleanor Audley,Verna Felton,Barbara Luddy,1.0,2.0,1.0,1.0,1.0,4.002,2.951,7.009,8.036,3.495,1930-04-05,1921-07-06,1905-11-19,1890-07-20,1908-05-25,enfant,moyen-âge,amour,dessin animé
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5766,694234,/a8dc8eyldtHMjrUluyLRKdVIhif.jpg,it,Pendant leurs vacances d'été sur les plages de...,7.660,/waOsLWxnqA0sa00QUpZCwGL0dsS.jpg,2020-07-01,Sous le soleil de Riccione,6.200,552,"[Comedy, Romance]",2,Antonio Usbergo,0.335,None,None,None,694234,Cristiano Caccamo,Saul Nanni,Andrea Roncato,Lorenzo Zurzolo,Ludovica Martino,2.0,2.0,2.0,2.0,1.0,6.585,4.948,1.634,28.534,4.751,1989-03-21,1999-02-04,1947-03-07,2000-03-21,1997-02-26,adulte,époque actuelle,amour,film d'auteur
5767,714888,/gh1Rghpf3BIISHSAw9GsObG4TN3.jpg,es,"Argentina, 1985 est inspiré de l'histoire vrai...",10.286,/ZjOG0RZlOM6YBiVdDbINadSxS4.jpg,2022-09-29,"Argentine, 1985",7.900,617,"[Drama, History, Crime]",2,Santiago Mitre,0.674,1980-12-04,None,"Bueno

In [7]:
# supprimer les colonnes non utilisées dans ML
df = df.drop(columns=['backdrop_path', 'overview', 'poster_path', 'movie_id', 'actor_birthday_1', 'actor_birthday_2',
       'actor_birthday_3', 'actor_birthday_4', 'actor_birthday_5'])

In [8]:
df.head(3)

,id_movie,original_language,popularity_movie,release_date,title,vote_average,vote_count,genre,gender_director,name_director,popularity_director,birthday_director,deathday_director,place_of_birth_director,actor_name_1,actor_name_2,actor_name_3,actor_name_4,actor_name_5,actor_gender_1,actor_gender_2,actor_gender_3,actor_gender_4,actor_gender_5,actor_popularity_1,actor_popularity_2,actor_popularity_3,actor_popularity_4,actor_popularity_5,Publique cible,Epoque,Sujet principal,Type
0,11224,en,79.005,1950-02-22,Cendrillon,7.041,6685,"[Family, Fantasy, Animation, Romance]",2,Hamilton Luske,2.479,1903-10-16,1968-02-19,"Chicago, Illinois, USA",Ilene Woods,Eleanor Audley,Verna Felton,Claire Du Brey,Rhoda Williams,1.0,1.0,1.0,1.0,1.0,5.250,7.009,8.036,4.592,2.626,enfant,moyen-âge,amour,dessin animé
1,389,en,58.940,1957-04-10,12 Hommes en colère,8.500,8655,[Drama],2,Sidney Lumet,4.824,1924-06-25,2011-04-09,"Philadelphia, Pennsylvania, USA",Martin Balsam,John Fiedler,Lee J. Cobb,E.G. Marshall,Jack Klugman,2.0,2.0,2.0,2.0,2.0,9.780,7.985,10.268,9.454,4.698,adulte,année 1950-1960,drame,film d'auteur
2,6844,en,56.194,1956-10-05,Les Dix Commandements,7.756,1603,"[Drama, History]",2,Cecil B. DeMille,6.705,1881-08-12,1959-01-21,"Ashfield, Massachusetts, USA",Charlton Heston,Yul Brynner,Anne Baxter,Edward G. Robinson,Yvonne De Carlo,2.0,2.0,1.0,2.0,1.0,22.823,8.561,9.878,9.671,15.539,adulte,antiquité,drame,blockbuster


In [9]:
df_movie = df.copy()

In [10]:
#df_movie[['release_date', 'birthday_director', 'deathday_director']] = df_movie[['release_date','birthday_director', 'deathday_director']].fillna(0)

date_list = ['release_date', 'birthday_director', 'deathday_director']
for date in date_list:
    try:
        df_movie[date] = pd.to_datetime(df_movie[date])
        df_movie[date] = ((df_movie[date].dt.year // 10) * 10)
    except: 
        pass

In [11]:
df_movie.head()

,id_movie,original_language,popularity_movie,release_date,title,vote_average,vote_count,genre,gender_director,name_director,popularity_director,birthday_director,deathday_director,place_of_birth_director,actor_name_1,actor_name_2,actor_name_3,actor_name_4,actor_name_5,actor_gender_1,actor_gender_2,actor_gender_3,actor_gender_4,actor_gender_5,actor_popularity_1,actor_popularity_2,actor_popularity_3,actor_popularity_4,actor_popularity_5,Publique cible,Epoque,Sujet principal,Type
0,11224,en,79.005,1950,Cendrillon,7.041,6685,"[Family, Fantasy, Animation, Romance]",2,Hamilton Luske,2.479,1900.0,1960.0,"Chicago, Illinois, USA",Ilene Woods,Eleanor Audley,Verna Felton,Claire Du Brey,Rhoda Williams,1.0,1.0,1.0,1.0,1.0,5.250,7.009,8.036,4.592,2.626,enfant,moyen-âge,amour,dessin animé
1,389,en,58.940,1950,12 Hommes en colère,8.500,8655,[Drama],2,Sidney Lumet,4.824,1920.0,2010.0,"Philadelphia, Pennsylvania, USA",Martin Balsam,John Fiedler,Lee J. Cobb,E.G. Marshall,Jack Klugman,2.0,2.0,2.0,2.0,2.0,9.780,7.985,10.268,9.454,4.698,adulte,année 1950-1960,drame,film d'auteur
2,6844,en,56.194,1950,Les Dix Commandements,7.756,1603,"[Drama, History]",2,Cecil B. DeMille,6.705,1880.0,1950.0,"Ashfield, Massachusetts, USA",Charlton Heston,Yul Brynner,Anne Baxter,Edward G. Robinson,Yvonne De Carlo,2.0,2.0,1.0,2.0,1.0,22.823,8.561,9.878,9.671,15.539,adulte,antiquité,drame,blockbuster
3,665,en,56.369,1950,Ben-Hur,7.881,2747,"[History, Drama, Adventure, Action]",2,William Wyler,4.382,1900.0,1980.0,"Mülhausen, Alsace-Lorraine, German Empire [now...",Charlton Heston,Stephen Boyd,Hugh Griffith,Jack Hawkins,Haya Harareet,2.0,2.0,2.0,2.0,1.0,22.823,8.798,4.333,9.360,2.899,adulte,antiquité,drame,blockbuster
4,10882,en,46.057,1950,La Belle au bois dormant,6.900,5123,"[Fantasy, Animation, Romance, Family]",2,Clyde Geronimi,3.546,1900.0,1980.0,"Chiavenna, Lombardy, Italy",Mary Costa,Bill Shirley,Eleanor Audley,Verna Felton,Barbara Luddy,1.0,2.0,1.0,1.0,1.0,4.002,2.951,7.009,8.036,3.495,enfant,moyen-âge,amour,dessin animé


In [12]:
df_movie[['release_date', 'birthday_director', 'deathday_director']] = df_movie[['release_date', 
                                                                                 'birthday_director', 'deathday_director']].fillna(0)
df_movie['birthday_director'] = df_movie['birthday_director'].astype(int)
df_movie['deathday_director'] = df_movie['deathday_director'].astype(int)
df_movie.head(3)

,id_movie,original_language,popularity_movie,release_date,title,vote_average,vote_count,genre,gender_director,name_director,popularity_director,birthday_director,deathday_director,place_of_birth_director,actor_name_1,actor_name_2,actor_name_3,actor_name_4,actor_name_5,actor_gender_1,actor_gender_2,actor_gender_3,actor_gender_4,actor_gender_5,actor_popularity_1,actor_popularity_2,actor_popularity_3,actor_popularity_4,actor_popularity_5,Publique cible,Epoque,Sujet principal,Type
0,11224,en,79.005,1950,Cendrillon,7.041,6685,"[Family, Fantasy, Animation, Romance]",2,Hamilton Luske,2.479,1900,1960,"Chicago, Illinois, USA",Ilene Woods,Eleanor Audley,Verna Felton,Claire Du Brey,Rhoda Williams,1.0,1.0,1.0,1.0,1.0,5.250,7.009,8.036,4.592,2.626,enfant,moyen-âge,amour,dessin animé
1,389,en,58.940,1950,12 Hommes en colère,8.500,8655,[Drama],2,Sidney Lumet,4.824,1920,2010,"Philadelphia, Pennsylvania, USA",Martin Balsam,John Fiedler,Lee J. Cobb,E.G. Marshall,Jack Klugman,2.0,2.0,2.0,2.0,2.0,9.780,7.985,10.268,9.454,4.698,adulte,année 1950-1960,drame,film d'auteur
2,6844,en,56.194,1950,Les Dix Commandements,7.756,1603,"[Drama, History]",2,Cecil B. DeMille,6.705,1880,1950,"Ashfield, Massachusetts, USA",Charlton Heston,Yul Brynner,Anne Baxter,Edward G. Robinson,Yvonne De Carlo,2.0,2.0,1.0,2.0,1.0,22.823,8.561,9.878,9.671,15.539,adulte,antiquité,drame,blockbuster


In [13]:
df_movie.columns

Index(['id_movie', 'original_language', 'popularity_movie', 'release_date',
       'title', 'vote_average', 'vote_count', 'genre', 'gender_director',
       'name_director', 'popularity_director', 'birthday_director',
       'deathday_director', 'place_of_birth_director', 'actor_name_1',
       'actor_name_2', 'actor_name_3', 'actor_name_4', 'actor_name_5',
       'actor_gender_1', 'actor_gender_2', 'actor_gender_3', 'actor_gender_4',
       'actor_gender_5', 'actor_popularity_1', 'actor_popularity_2',
       'actor_popularity_3', 'actor_popularity_4', 'actor_popularity_5',
       'Publique cible', 'Epoque', 'Sujet principal', 'Type'],
      dtype='object')

In [14]:
actor_list = []
for col in range(1, 6): 
    acteur_name_col = f"actor_name_{col}"
    for i in range(0, len(df_movie)): 
        acteur_name = df_movie.loc[i, acteur_name_col]
        if acteur_name not in actor_list: 
            actor_list.append(acteur_name)

In [15]:
resultat = {}

for acteur in actor_list: 
    index_list = []
    popularity_list = []
    for col in range(1, 6): 
        acteur_name_col = f"actor_name_{col}"
        
        index = df_movie[df_movie[acteur_name_col] == acteur].index
        for i in index : 
            index_list.append(i)

    for index in index_list:
        popularity = df_movie.loc[index, 'popularity_movie']
        popularity_list.append(popularity)
    
    try: 
        resultat[acteur] = float(round(sum(popularity_list) / len(popularity_list),2))
    except: 
        pass


In [16]:
resultat

{'Ilene Woods': 79.0,
 'Martin Balsam': 27.72,
 'Charlton Heston': 29.45,
 'Mary Costa': 46.06,
 'Kathryn Beaumont': 48.05,
 'Bobby Driscoll': 47.29,
 'Barbara Luddy': 43.34,
 'Toshirō Mifune': 15.8,
 'James Stewart': 19.52,
 'Tony Curtis': 29.45,
 'Cary Grant': 24.5,
 'John Wayne': 20.14,
 'Gene Kelly': 15.7,
 'Michael Rennie': 24.04,
 'Audrey Hepburn': 17.59,
 'Kirk Douglas': 22.81,
 'Akira Takarada': 19.72,
 'William Holden': 16.71,
 'Gene Barry': 15.89,
 'Alan Ladd': 22.12,
 'Robert Mitchum': 21.98,
 'Tyrone Power': 21.72,
 'Walter Pidgeon': 25.16,
 'Vivien Leigh': 21.84,
 'Ray Milland': 21.13,
 'Humphrey Bogart': 15.34,
 'Marlon Brando': 43.06,
 'Max von Sydow': 31.4,
 'Elizabeth Taylor': 16.82,
 'Steve McQueen': 21.37,
 'Takashi Shimura': 18.97,
 'Yves Montand': 13.89,
 'Emmanuelle Riva': 21.89,
 'Kevin McCarthy': 15.34,
 'Marilyn Monroe': 17.54,
 'David Hedison': 13.85,
 'David Niven': 17.46,
 'Véra Clouzot': 17.84,
 'Burt Lancaster': 17.5,
 'James Dean': 16.72,
 'Farley Granger

In [17]:
print(type(resultat.get(df_movie.loc[0, 'actor_name_1'])))

<class 'float'>


In [18]:
for col in range(1, 6): 
    acteur_name_col = f"actor_name_{col}"
    acteur_score_col = f"actor_score_{col}"
    for i in range(0, len(df_movie)):
        df_movie.loc[i, acteur_score_col] = resultat.get(df_movie.loc[i, acteur_name_col])

In [19]:
df_movie.head()

,id_movie,original_language,popularity_movie,release_date,title,vote_average,vote_count,genre,gender_director,name_director,popularity_director,birthday_director,deathday_director,place_of_birth_director,actor_name_1,actor_name_2,actor_name_3,actor_name_4,actor_name_5,actor_gender_1,actor_gender_2,actor_gender_3,actor_gender_4,actor_gender_5,actor_popularity_1,actor_popularity_2,actor_popularity_3,actor_popularity_4,actor_popularity_5,Publique cible,Epoque,Sujet principal,Type,actor_score_1,actor_score_2,actor_score_3,actor_score_4,actor_score_5
0,11224,en,79.005,1950,Cendrillon,7.041,6685,"[Family, Fantasy, Animation, Romance]",2,Hamilton Luske,2.479,1900,1960,"Chicago, Illinois, USA",Ilene Woods,Eleanor Audley,Verna Felton,Claire Du Brey,Rhoda Williams,1.0,1.0,1.0,1.0,1.0,5.250,7.009,8.036,4.592,2.626,enfant,moyen-âge,amour,dessin animé,79.00,62.53,62.53,79.00,79.00
1,389,en,58.940,1950,12 Hommes en colère,8.500,8655,[Drama],2,Sidney Lumet,4.824,1920,2010,"Philadelphia, Pennsylvania, USA",Martin Balsam,John Fiedler,Lee J. Cobb,E.G. Marshall,Jack Klugman,2.0,2.0,2.0,2.0,2.0,9.780,7.985,10.268,9.454,4.698,adulte,année 1950-1960,drame,film d'auteur,27.72,27.40,44.92,58.94,58.94
2,6844,en,56.194,1950,Les Dix Commandements,7.756,1603,"[Drama, History]",2,Cecil B. DeMille,6.705,1880,1950,"Ashfield, Massachusetts, USA",Charlton Heston,Yul Brynner,Anne Baxter,Edward G. Robinson,Yvonne De Carlo,2.0,2.0,1.0,2.0,1.0,22.823,8.561,9.878,9.671,15.539,adulte,antiquité,drame,blockbuster,29.45,38.47,56.19,56.19,56.19
3,665,en,56.369,1950,Ben-Hur,7.881,2747,"[History, Drama, Adventure, Action]",2,William Wyler,4.382,1900,1980,"Mülhausen, Alsace-Lorraine, German Empire [now...",Charlton Heston,Stephen Boyd,Hugh Griffith,Jack Hawkins,Haya Harareet,2.0,2.0,2.0,2.0,1.0,22.823,8.798,4.333,9.360,2.899,adulte,antiquité,drame,blockbuster,29.45,56.37,33.52,37.68,56.37
4,10882,en,46.057,1950,La Belle au bois dormant,6.900,5123,"[Fantasy, Animation, Romance, Family]",2,Clyde Geronimi,3.546,1900,1980,"Chiavenna, Lombardy, Italy",Mary Costa,Bill Shirley,Eleanor Audley,Verna Felton,Barbara Luddy,1.0,2.0,1.0,1.0,1.0,4.002,2.951,7.009,8.036,3.495,enfant,moyen-âge,amour,dessin animé,46.06,46.06,62.53,62.53,43.34


In [22]:
# Pour rempaler les noms des acteurs par score moyen : 
#for col in range(1, 6): 
#    acteur_name_col = f"actor_name_{col}"
#    for i in range(0, len(df_movie)): 
#        df_movie.loc[i, acteur_name_col] = resultat.get(df_movie.loc[i, acteur_name_col])



In [11]:
#df_movie = df_movie.rename(columns={'actor_name_1': 'actor_1_mean_movie', 
#                         'actor_name_2': 'actor_2_mean_movie', 
#                         'actor_name_3': 'actor_3_mean_movie',
#                         'actor_name_4': 'actor_4_mean_movie', 
#                         'actor_name_5': 'actor_5_mean_movie'
#})

In [20]:
# Supprimer le gender des acteurs 
df_movie.drop(columns=['actor_gender_1', 'actor_gender_2', 'actor_gender_3', 'actor_gender_4', 'actor_gender_5'], inplace=True)


In [21]:
df_movie.head()

,id_movie,original_language,popularity_movie,release_date,title,vote_average,vote_count,genre,gender_director,name_director,popularity_director,birthday_director,deathday_director,place_of_birth_director,actor_name_1,actor_name_2,actor_name_3,actor_name_4,actor_name_5,actor_popularity_1,actor_popularity_2,actor_popularity_3,actor_popularity_4,actor_popularity_5,Publique cible,Epoque,Sujet principal,Type,actor_score_1,actor_score_2,actor_score_3,actor_score_4,actor_score_5
0,11224,en,79.005,1950,Cendrillon,7.041,6685,"[Family, Fantasy, Animation, Romance]",2,Hamilton Luske,2.479,1900,1960,"Chicago, Illinois, USA",Ilene Woods,Eleanor Audley,Verna Felton,Claire Du Brey,Rhoda Williams,5.250,7.009,8.036,4.592,2.626,enfant,moyen-âge,amour,dessin animé,79.00,62.53,62.53,79.00,79.00
1,389,en,58.940,1950,12 Hommes en colère,8.500,8655,[Drama],2,Sidney Lumet,4.824,1920,2010,"Philadelphia, Pennsylvania, USA",Martin Balsam,John Fiedler,Lee J. Cobb,E.G. Marshall,Jack Klugman,9.780,7.985,10.268,9.454,4.698,adulte,année 1950-1960,drame,film d'auteur,27.72,27.40,44.92,58.94,58.94
2,6844,en,56.194,1950,Les Dix Commandements,7.756,1603,"[Drama, History]",2,Cecil B. DeMille,6.705,1880,1950,"Ashfield, Massachusetts, USA",Charlton Heston,Yul Brynner,Anne Baxter,Edward G. Robinson,Yvonne De Carlo,22.823,8.561,9.878,9.671,15.539,adulte,antiquité,drame,blockbuster,29.45,38.47,56.19,56.19,56.19
3,665,en,56.369,1950,Ben-Hur,7.881,2747,"[History, Drama, Adventure, Action]",2,William Wyler,4.382,1900,1980,"Mülhausen, Alsace-Lorraine, German Empire [now...",Charlton Heston,Stephen Boyd,Hugh Griffith,Jack Hawkins,Haya Harareet,22.823,8.798,4.333,9.360,2.899,adulte,antiquité,drame,blockbuster,29.45,56.37,33.52,37.68,56.37
4,10882,en,46.057,1950,La Belle au bois dormant,6.900,5123,"[Fantasy, Animation, Romance, Family]",2,Clyde Geronimi,3.546,1900,1980,"Chiavenna, Lombardy, Italy",Mary Costa,Bill Shirley,Eleanor Audley,Verna Felton,Barbara Luddy,4.002,2.951,7.009,8.036,3.495,enfant,moyen-âge,amour,dessin animé,46.06,46.06,62.53,62.53,43.34


In [22]:
df_movie.isnull().sum()

id_movie                     0
original_language            0
popularity_movie             0
release_date                 0
title                        0
vote_average                 0
vote_count                   0
genre                        0
gender_director              0
name_director                0
popularity_director          0
birthday_director            0
deathday_director            0
place_of_birth_director    435
actor_name_1                 0
actor_name_2                 3
actor_name_3                10
actor_name_4                33
actor_name_5               242
actor_popularity_1           0
actor_popularity_2           3
actor_popularity_3          10
actor_popularity_4          33
actor_popularity_5         242
Publique cible               0
Epoque                       0
Sujet principal              0
Type                         0
actor_score_1                0
actor_score_2                3
actor_score_3               10
actor_score_4               33
actor_sc

In [23]:
# Remplir les valeurs nulles dans d'autres colonnes numériques
df_movie[['actor_popularity_2', 'actor_popularity_3', 'actor_popularity_4', 
          'actor_popularity_5', 'actor_score_2', 'actor_score_3', 
          'actor_score_4', 'actor_score_5']] = df_movie[['actor_popularity_2', 'actor_popularity_3', 
                                                         'actor_popularity_4', 'actor_popularity_5', 
                                                         'actor_score_2', 'actor_score_3', 'actor_score_4', 'actor_score_5']].fillna(0)



# Remplir les valeurs nulls dans d'autres colonnes non-numériques : 
df_movie[['place_of_birth_director','actor_name_2', 
          'actor_name_3', 'actor_name_4', 'actor_name_5']] = df_movie[['place_of_birth_director','actor_name_2', 
                                                                       'actor_name_3', 'actor_name_4', 'actor_name_5']].fillna('aucun')



In [24]:
df_movie.isnull().sum()

id_movie                   0
original_language          0
popularity_movie           0
release_date               0
title                      0
vote_average               0
vote_count                 0
genre                      0
gender_director            0
name_director              0
popularity_director        0
birthday_director          0
deathday_director          0
place_of_birth_director    0
actor_name_1               0
actor_name_2               0
actor_name_3               0
actor_name_4               0
actor_name_5               0
actor_popularity_1         0
actor_popularity_2         0
actor_popularity_3         0
actor_popularity_4         0
actor_popularity_5         0
Publique cible             0
Epoque                     0
Sujet principal            0
Type                       0
actor_score_1              0
actor_score_2              0
actor_score_3              0
actor_score_4              0
actor_score_5              0
dtype: int64

In [31]:
df_movie.shape

(5771, 33)

In [81]:
#df_movie.to_parquet("df_movie.parquet")

In [33]:
df_movie.columns

Index(['id_movie', 'original_language', 'popularity_movie', 'release_date',
       'title', 'vote_average', 'vote_count', 'genre', 'gender_director',
       'name_director', 'popularity_director', 'birthday_director',
       'deathday_director', 'place_of_birth_director', 'actor_name_1',
       'actor_name_2', 'actor_name_3', 'actor_name_4', 'actor_name_5',
       'actor_popularity_1', 'actor_popularity_2', 'actor_popularity_3',
       'actor_popularity_4', 'actor_popularity_5', 'Publique cible', 'Epoque',
       'Sujet principal', 'Type', 'actor_score_1', 'actor_score_2',
       'actor_score_3', 'actor_score_4', 'actor_score_5'],
      dtype='object')

ne pas utiliser pour la ML : id_movie, 'gender_director', 'birthday_director',
       'deathday_director', title

'actor_popularity_1', 'actor_popularity_2', 'actor_popularity_3',
       'actor_popularity_4', 'actor_popularity_5'

       'actor_name_1',
       'actor_name_2', 'actor_name_3', 'actor_name_4', 'actor_name_5'

    'actor_score_1', 'actor_score_2',
       'actor_score_3', 'actor_score_4', 'actor_score_5'

In [52]:
numeric_cols = ['popularity_movie', 'vote_average', 'vote_count',
       'popularity_director']


categorical_cols = ['original_language', 'name_director', 'place_of_birth_director', 'Publique cible', 'Epoque',
       'Sujet principal', 'Type', 'actor_name_1', 'actor_name_2', 'actor_name_3', 'actor_name_4', 'actor_name_5']


binarizer_cols = ['genre']

date_cols = ['release_date']

In [53]:
# Custom transformer for MultiLabelBinarizer
class MultiLabelBinarizerPipelineFriendly(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.mlb = MultiLabelBinarizer()

    def fit(self, X, y=None):
        self.mlb.fit(X)
        return self

    def transform(self, X):
        return self.mlb.transform(X)

    def get_feature_names_out(self, input_features=None):
        return self.mlb.classes_

In [55]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

date_transformer = Pipeline(steps=[
    ('date', OrdinalEncoder())
])

# On combine tout dans un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols),
        ('genres', MultiLabelBinarizerPipelineFriendly(), 'genre'),
        ('date', date_transformer, date_cols)
    ]
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

In [56]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['popularity_movie',
                                                   'vote_average', 'vote_count',
                                                   'popularity_director']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['original_language',
                                                   'name_director',
                                                   'place_of_birth_director',
                                                   'Publique cible', 'Epoque',
                                                   'Sujet principal', 'Type',
                                                   'actor_name_1',
                                                   'actor_name_2',
                                                   'actor_name_3',
                                                   'actor_name_4',
                                                   'actor_name_5']),
                                                 ('genres',
                                                  MultiLabelBinarizerPipelineFriendly(),
                                                  'genre'),
                                                 ('date',
                                                  Pipeline(steps=[('date',
                                                                   OrdinalEncoder())]),
                                                  ['release_date'])]))])

In [57]:
# Transformation des données
processed_data = pipeline.fit_transform(df_movie)

# Affichage des données transformées
transformed_df = pd.DataFrame(processed_data.todense(), columns=pipeline["preprocessor"].get_feature_names_out())

In [58]:
processed_data.todense().shape

(5771, 21803)

In [59]:
# Initialisation du modèle KNN
knn_model = NearestNeighbors(n_neighbors=6, metric='manhattan')  # 5 voisins les plus proches hormis le point de référence lui meme
knn_model.fit(processed_data)

NearestNeighbors(metric='manhattan', n_neighbors=6)

In [61]:
# Exemple de recommandation pour un item
def recommend_similar_films(title, 
                              data: pd.DataFrame=processed_data, 
                              model: NearestNeighbors=knn_model, 
                              original_data: pd.DataFrame=df_movie, 
                              n_neighbors: int=6) -> tuple[list[float], list[float]]:
    """
    Trouve les n éléments les plus proches pour un élément donné.

    Args:
        title: le titre d'un film
        data: Données transformées utilisées pour KNN = df processed data
        model: Modèle KNN pré-entraîné.
        original_data: Données originales (pour affichage) = df movies
        n_neighbors: Nombre de voisins à recommander = 6 pour avoir 5 recommandations

    Returns:
        films similaires: Indices et distances des filmes similaires.
    """
    index = original_data[original_data['title'] == title].index[0] # l'indice du filme recherché 
    distances, indices = model.kneighbors(data[index], n_neighbors=n_neighbors) 
    print(f"Le filme recherché : {title}:\n")
    display(pd.DataFrame(original_data.iloc[index]).T)
    display(distances[0])
    print(f"Recommandations pour le film : {title} :\n")
    display(original_data.iloc[indices[0][1:]])


In [62]:
recommend_similar_films("Vice-versa 2")

Le filme recherché : Vice-versa 2:



,id_movie,original_language,popularity_movie,release_date,title,vote_average,vote_count,genre,gender_director,name_director,popularity_director,birthday_director,deathday_director,place_of_birth_director,actor_name_1,actor_name_2,actor_name_3,actor_name_4,actor_name_5,actor_popularity_1,actor_popularity_2,actor_popularity_3,actor_popularity_4,actor_popularity_5,Publique cible,Epoque,Sujet principal,Type,actor_score_1,actor_score_2,actor_score_3,actor_score_4,actor_score_5
5049,1022789,en,596.256,2020,Vice-versa 2,7.588,4833,[Animation],2,Kelsey Mann,7.164,0,0,aucun,Amy Poehler,Maya Hawke,Kensington Tallman,Liza Lapira,Tony Hale,12.713,18.329,8.671,12.547,19.534,enfant,époque actuelle,humour,dessin animé,117.61,596.26,596.26,311.08,174.38


array([ 0.        , 19.92036145, 23.93001426, 23.95557749, 24.48021806,
       24.49992018])

Recommandations pour le film : Vice-versa 2 :



,id_movie,original_language,popularity_movie,release_date,title,vote_average,vote_count,genre,gender_director,name_director,popularity_director,birthday_director,deathday_director,place_of_birth_director,actor_name_1,actor_name_2,actor_name_3,actor_name_4,actor_name_5,actor_popularity_1,actor_popularity_2,actor_popularity_3,actor_popularity_4,actor_popularity_5,Publique cible,Epoque,Sujet principal,Type,actor_score_1,actor_score_2,actor_score_3,actor_score_4,actor_score_5
5048,519182,en,698.614,2020,"Moi, moche et méchant 4",7.079,2209,"[Animation, Family, Comedy, Action]",2,Chris Renaud,6.794,1960,0,"Baltimore, Maryland, USA",Steve Carell,Kristen Wiig,Will Ferrell,Sofía Vergara,Miranda Cosgrove,28.442,36.424,29.311,41.889,22.581,enfant,époque actuelle,humour,dessin animé,60.02,60.63,51.56,245.70,156.91
5179,585511,en,70.015,2020,Luck,7.771,1699,"[Animation, Adventure, Comedy, Fantasy]",1,Peggy Holmes,3.793,0,0,aucun,Eva Noblezada,Simon Pegg,Jane Fonda,Whoopi Goldberg,Flula Borg,6.352,30.535,16.109,39.410,14.090,enfant,époque actuelle,humour,dessin animé,70.02,38.77,23.10,29.18,70.02
3473,44874,en,23.767,2010,Barbie : La Magie de la mode,7.200,752,"[Family, Animation]",2,William Lau,5.516,0,0,aucun,Diana Kaarina,Adrian Petriw,Tabitha St. Germain,Patricia Drake,Alexa Devine,5.855,4.459,6.145,2.641,0.828,enfant,époque actuelle,humour,dessin animé,34.75,23.77,19.84,23.77,23.77
3126,73456,en,45.737,2010,Barbie apprentie Princesse,7.500,809,"[Family, Animation, Fantasy]",2,Ezekiel Norton,2.866,0,0,aucun,Diana Kaarina,Morwenna Banks,Nicole Oliver,Brittney Wilson,Ali Liebert,5.855,7.590,6.397,6.036,9.154,enfant,époque actuelle,humour,dessin animé,34.75,34.96,30.00,45.74,45.74
4152,129533,en,17.492,2010,Barbie : La Princesse et la popstar,7.100,592,"[Family, Animation]",2,Ezekiel Norton,2.866,0,0,aucun,Kelly Sheridan,Jennifer Waris,Ashleigh Ball,Tiffany Giardina,Ellie King,6.202,1.182,9.660,0.203,1.506,enfant,époque actuelle,humour,dessin animé,26.31,17.49,17.49,17.49,17.49


In [51]:
df_movie

,id_movie,original_language,popularity_movie,release_date,title,vote_average,vote_count,genre,gender_director,name_director,popularity_director,birthday_director,deathday_director,place_of_birth_director,actor_name_1,actor_name_2,actor_name_3,actor_name_4,actor_name_5,actor_popularity_1,actor_popularity_2,actor_popularity_3,actor_popularity_4,actor_popularity_5,Publique cible,Epoque,Sujet principal,Type,actor_score_1,actor_score_2,actor_score_3,actor_score_4,actor_score_5
0,11224,en,79.005,1950,Cendrillon,7.041,6685,"[Family, Fantasy, Animation, Romance]",2,Hamilton Luske,2.479,1900,1960,"Chicago, Illinois, USA",Ilene Woods,Eleanor Audley,Verna Felton,Claire Du Brey,Rhoda Williams,5.250,7.009,8.036,4.592,2.626,enfant,moyen-âge,amour,dessin animé,79.00,62.53,62.53,79.00,79.00
1,389,en,58.940,1950,12 Hommes en colère,8.500,8655,[Drama],2,Sidney Lumet,4.824,1920,2010,"Philadelphia, Pennsylvania, USA",Martin Balsam,John Fiedler,Lee J. Cobb,E.G. Marshall,Jack Klugman,9.780,7.985,10.268,9.454,4.698,adulte,année 1950-1960,drame,film d'auteur,27.72,27.40,44.92,58.94,58.94
2,6844,en,56.194,1950,Les Dix Commandements,7.756,1603,"[Drama, History]",2,Cecil B. DeMille,6.705,1880,1950,"Ashfield, Massachusetts, USA",Charlton Heston,Yul Brynner,Anne Baxter,Edward G. Robinson,Yvonne De Carlo,22.823,8.561,9.878,9.671,15.539,adulte,antiquité,drame,blockbuster,29.45,38.47,56.19,56.19,56.19
3,665,en,56.369,1950,Ben-Hur,7.881,2747,"[History, Drama, Adventure, Action]",2,William Wyler,4.382,1900,1980,"Mülhausen, Alsace-Lorraine, German Empire [now...",Charlton Heston,Stephen Boyd,Hugh Griffith,Jack Hawkins,Haya Harareet,22.823,8.798,4.333,9.360,2.899,adulte,antiquité,drame,blockbuster,29.45,56.37,33.52,37.68,56.37
4,10882,en,46.057,1950,La Belle au bois dormant,6.900,5123,"[Fantasy, Animation, Romance, Family]",2,Clyde Geronimi,3.546,1900,1980,"Chiavenna, Lombardy, Italy",Mary Costa,Bill Shirley,Eleanor Audley,Verna Felton,Barbara Luddy,4.002,2.951,7.009,8.036,3.495,enfant,moyen-âge,amour,dessin animé,46.06,46.06,62.53,62.53,43.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5766,694234,it,7.660,2020,Sous le soleil de Riccione,6.200,552,"[Comedy, Romance]",2,Antonio Usbergo,0.335,0,0,aucun,Cristiano Caccamo,Saul Nanni,Andrea Roncato,Lorenzo Zurzolo,Ludovica Martino,6.585,4.948,1.634,28.534,4.751,adulte,époque actuelle,amour,film d'auteur,7.66,7.66,7.66,7.66,7.66
5767,714888,es,10.286,2020,"Argentine, 1985",7.900,617,"[Drama, History, Crime]",2,Santiago Mitre,0.674,1980,0,"Buenos Aires, Argentina",Ricardo Darín,Peter Lanzani,Alejandra Flechner,Paula Ransenberg,Carlos Portaluppi,5.877,3.067,1.376,0.346,1.270,adulte,années 1970-1980,biopic,film d'auteur,15.46,10.29,10.29,10.29,10.29
5768,606954,it,6.531,2020,Odio l'estate,7.100,921,[Comedy],2,Massimo Venier,3.630,1960,0,"Varese, Italy",Aldo Baglio,Giovanni Storti,Giacomo Poretti,Lucia Mascino,Carlotta Natoli,3.680,2.184,1.579,5.234,2.985,adulte,époque actuelle,drame,film d'auteur,5.85,5.85,5.85,6.53,6.53
5769,614292,it,6.972,2020,Storia di vacanze,6.742,567,[Drama],2,Damiano D'Innocenzo,1.030,1980,0,"Rome, Lazio, Italy",Elio Germano,Tommaso Di Cola,Giulietta Rebeggiani,Gabriel Montesi,Justin Alexander Korovkin,4.840,0.662,1.511,1.177,0.636,adulte,époque actuelle,drame,film d'auteur,8.03,6.97,6.97,6.97,6.97


In [46]:
df1['title'].sample(10)

2754                       Severance
61      La Nuit de tous les mystères
2333    Fast & Furious : Tokyo drift
5607                   The Silencing
4032      Des saumons dans le désert
4424                        99 Homes
3077                   Suicide Squad
4772                       Macgruber
4834              Belle et Sébastien
267           Rocky II : La Revanche
Name: title, dtype: object

In [47]:
df1.loc[1528, 'title']

'Pirates des Caraïbes\xa0: Le Secret du coffre maudit'

Fast and Furious